# Preprocess the daycare data and save biom tables and mapping files
We load the data from 2 runs, aggregate, normalize/rarify, join, and add qiime generated taxonomy.

In [1]:
import calour as ca
import calour_utils as cu

failed to load logging config file


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import numpy as np
import matplotlib as mpl
import pandas as pd
import scipy as sp;


In [3]:
pwd

'/Users/amnon/git/paper-daycare'

In [4]:
ca.set_log_level(11)

In [5]:
%matplotlib inline

In [6]:
db=ca.database._get_database_class('dbbact')

creating logger


# NOTE: since the runs contain other samples as well, the run output is not included in the git repo
## We have saved in the git repo the resulting joined tables

# Load the data
## we have 2 runs, with some samples duplicated between the two runs
## so we load without normalization/filtering, then aggregate by sum and then filter

In [ ]:
ca.set_log_level('ERROR')
dat=ca.read_amplicon('data/16S_DB1-10.biom','data/DB7_10_11_ganim_amnon_map_14July.txt',normalize=None,min_reads=500)
ca.set_log_level('INFO')

In [8]:
ca.set_log_level('ERROR')
dat2=ca.read_amplicon('data/all.db11.biom','data/DB7_10_11_ganim_amnon_map_14July.txt',normalize=None,min_reads=500)
ca.set_log_level('INFO')

## get rid of samples not in mapping file


In [9]:
dat=dat.filter_samples('BarcodeSequence',None)
dat2=dat2.filter_samples('BarcodeSequence',None)

## combine the 2 runs

In [10]:
dat=dat.join_experiments(dat2,'orig_run')

In [11]:
dat

AmpliconExperiment ("join  & ") with 271 samples, 14052 features

## Join samples with same sample_ID (i.e. rerun of same sample in the 2 runs)

In [12]:
dat=dat.aggregate_by_metadata('sample_ID',agg='mean')
dat

AmpliconExperiment ("join  & ") with 268 samples, 14052 features

## Get rid of non-child samples (family members)

In [13]:
dat=dat.filter_samples('kindergarten','Family',negate=True)
dat

AmpliconExperiment ("join  & ") with 268 samples, 14052 features

## Sort by timepoint, and within each timepoint by subjectID

In [38]:
dat=dat.sort_samples('pn_ID')
dat=dat.sort_samples('Time')
dat=dat.sort_samples('kindergarten')

## Save the non-normalized/rarified table

In [39]:
dat.save('data/gan-joined')

2020-08-16 16:12:34 INFO Metadata field taxonomy not found. Saving biom table without metadata


# *** Analysis can be recreated from this point

## Reload the data and add the qiime2 generated taxonomy

In [7]:
ca.set_log_level('ERROR')
dat=ca.read_amplicon('data/gan-joined.biom', sample_metadata_file='data/gan-joined_sample.txt', feature_metadata_file='data/taxonomy-all.tsv',
                     min_reads=0, normalize=None)
ca.set_log_level('INFO')
dat.feature_metadata['taxonomy']=dat.feature_metadata['Taxon']
dat=dat.split_taxonomy()

# get rid of samples with not enough reads
## How many samples do we lose for different read-depths?

In [8]:
nreads=np.sum(dat.get_data(sparse=False),axis=1)

In [9]:
for i in range(8):
    print('min reads %d: num samples deleted %d' % (i*1000, np.sum(nreads<i*1000)))

min reads 0: num samples deleted 0
min reads 1000: num samples deleted 0
min reads 2000: num samples deleted 0
min reads 3000: num samples deleted 0
min reads 4000: num samples deleted 0
min reads 5000: num samples deleted 6
min reads 6000: num samples deleted 11
min reads 7000: num samples deleted 22


In [10]:
dat

AmpliconExperiment with 268 samples, 14052 features

### We choose the threshold to be 4000 (so we lose 0 samples)

In [11]:
dat=dat.reorder(nreads>=4000,axis='s')

# Prepare the normalized biom table

## Normalize data to percentile (total 100/sample)
This is done by total sum scaling (TSS) - multiplication by a constant per sample

In [12]:
gan=dat.normalize(100)

## Get rid of features with <0.1% reads total over all samples, and cluster the features

In [13]:
gan=gan.filter_sum_abundance(0.1)
gan=gan.cluster_features()

2021-10-04 09:52:09 INFO After filtering, 928 remain.
2021-10-04 09:52:09 INFO After filtering, 928 remain.


In [14]:
gan

AmpliconExperiment with 268 samples, 928 features

## Save the normalized biom table

In [15]:
gan.save('data/gan-normalized')

### and save the fasta file

In [16]:
gan.save_fasta('data/gan-normalized.fa')

# Prepare the subsampled (rarified) table for the alpha-diversity

## Subsample to 4000 reads/sample

In [20]:
dat.data=dat.data.astype(int)
dat_subsampled=dat.subsample_count(4000, random_seed=2021)

In [21]:
dat_subsampled.sample_metadata['numSpecies']=np.sum(dat_subsampled.data>0,axis=1)

In [22]:
dat_subsampled

AmpliconExperiment with 268 samples, 14052 features

In [23]:
dat_subsampled.save('data/gan-subsampled')

# Create 10 rarefactions for testing the effect of specific rarefactions on the results

In [24]:
for i in range(10):
    dat.data=dat.data.astype(int)
    dat_subsampled=dat.subsample_count(4000, random_seed=1000+i)
    dat_subsampled.sample_metadata['numSpecies']=np.sum(dat_subsampled.data>0,axis=1)
    dat_subsampled.save('./data/rarefactions/gan-subsampled-%s' % i)